### Problem 2

In [23]:
%%file reaction_coeffs.py
def k_const(k):
    """Returns the constant reaction rate coefficients
    
    INPUTS
    ======
    k: float, constant reaction rate coefficient
    
    RETURNS
    =======
    constant reaction rate coefficients, which is k
    
    EXAMPLES
    ========
    >>> k_const(1.5)
    1.5
    """
    k1 = k
    return k1

def k_arr(A, E, T, R=8.314):
    """Returns Arrhenius reaction rate coefficients
    
    INPUTS
    ======
    A: float, the Arrhenius prefactor, strictly positive
    E: float, a parameter
    T: float, temperature using a Kelvin scale, strictly positive
    R: float, default value is 8.314 which is the ideal gas constant. 
        R should not be changed except to convert units
    
    RETURNS
    =======
    Arrhenius reaction rate coefficients: a float
        a ValueError exception is raised if A <= 0
        a ValueError exception is raised if T <= 0
    Raise OverflowError if the result is beyond the maximum representable finite float
    Raise UnderflowError if the result is below the minimum positive normalized float
        since A, T and exponential function are all strictly positive,
        only consider positive overflow and positive underflow
    
    EXAMPLES
    ========
    >>>k_arr(10**7, 10**3, 10**2)
    3003549.08896
    """
    import numpy as np
    import sys
    if A <= 0:
        raise ValueError("The Arrhenius prefactor must be strictly positive.")
    if T <= 0:
        raise ValueError("Temperature must be strictly positive.")
    k2 = A * np.exp(-E / (R*T))
    if k2 > sys.float_info.max:
        raise OverflowError("Arrhenius coefficients overflow.")
    if k2 < sys.float_info.min:
        raise UnderflowError("Arrhenius coefficients underflow.")
    return k2

def k_modarr(A, b, E, T, R=8.314):
    """Returns the modified Arrhenius reaction rate coefficients.
    
    INPUTS
    ======
    A: float, the Arrhenius prefactor, strictly positive
    b: float, a parameter, must be real
    E: float, a parameter
    T: float, temperature using a Kelvin scale, strictly positive
    R: float, default value is 8.314 which is the ideal gas constant. 
        R should not be changed except to convert units
    
    RETURNS
    =======
    Modified Arrhenius reaction rate coefficients: a float
        a ValueError exception is raised if A <= 0
        a ValueError exception is raised if T <= 0
        a ValueError exception is raised if b is not real.
    Raise OverflowError if the result is beyond the maximum representable finite float
    Raise UnderflowError if the result is below the minimum positive normalized float
        since A, T and exponential function are all strictly positive,
        only consider positive overflow and positive underflow
    
    EXAMPLES
    ========
    >>>k_modarr(10**7, 0.5, 10**3, 10**2)
    30035490.889639609
    """
    import numpy as np
    if A <= 0:
        raise ValueError("The Arrhenius prefactor must be strictly positive.")
    if T <= 0:
        raise ValueError("Temperature must be strictly positive.")
    if np.iscomplex(b):
        raise ValueError("The parameter b must be real.")
    k3 = A * (T**b) * np.exp(-E / (R*T))
    if k3 > sys.float_info.max:
        raise OverflowError("Arrhenius coefficients overflow.")
    if k3 < sys.float_info.min:
        raise UnderflowError("Arrhenius coefficients underflow.")
    return k3

Writing reaction_coeffs.py


### Problem 3

In [31]:
def progress_rate1(v_i, x, k):
    """Returns the progress rate for a reaction (v_A)A+(v_B)B --> (v_C)C
    
    INPUTS
    ====== 
    v_i: a list of floats representing Stoichiometric coefficients of reactants
        ordered according to the reaction form, left to right
    x: a list of floats representing the concentration of each reactant and product
        ordered according to the reaction form, left to right
    k: float, reaction rate coefficient
        
    RETURNS
    =======
    progress rate for the reaction: a float
    
    EXAMPLES
    ========
    >>>progress_rate1([2.0, 1.0, 1.0], [1.0, 2.0, 3.0], 10)
    60.0
    """
    r=1
    for v, xi in zip(v_i, x):
        r *= xi**v
    w = k*r
    return w
    

In [32]:
progress_rate1([2.0,1.0,1.0], [1.0,2.0,3.0], 10)

60.0

### Problem 4

In [54]:
def progress_rate2(v_ij1, v_ij2, x, k):
    """Returns the progress rate for a system of reactions of the following form:
        (v_11')A + (v_21')B --> (v_31'')C
        (v_12')A + (v_32')C --> (v_22'')B + (v_32'')C

    INPUTS
    =======
    v_ij1: a numpy array of floats, representing the stoichiometric coefficient of reactnat i in reaction j
    v_ij2: a numpy array of floats, representing the stoichiometric coefficient of product i in reactio j
    x: a list of floats representing the concentration of each reactant and product
    k: float, reaction rate coefficient
    
    RETURNS
    =======
    progress rate for the system of reactions: a list of floats [w1, w2, ..., wj]
        wj is the progress rate for reaction j
    
    EXAMPLES:
    >>> progress_rate2(np.array([[1.0, 2.0, 0.0],[2.0, 0.0, 2.0]]),np.array([[0.0, 0.0, 2.0],
                        [0.0, 1.0, 1.0]]), [1.0,2.0,1.0], 10)
    [40.0, 10.0]

"""
    import numpy as np
    v_i1 = list(v_ij1[:][0])
    v_i2 = list(v_ij1[:][1])
    w=[]
    r1 = 1
    r2 = 1
    for v1, xi in zip(v_i1, x):
        r1 *= xi**v1
    w1 = k*r1
    w.append(w1)
    for v2, xi in zip(v_i2, x):
        r2 *= xi**v2
    w2 = k*r2
    w.append(w2)
    return w

In [55]:
import numpy as np
v_ij1 = np.array([[1.0, 2.0, 0.0], 
                 [2.0, 0.0, 2.0]])
v_ij2 = np.array([[0.0, 0.0, 2.0], 
                 [0.0, 1.0, 1.0]])
v_i1 = v_ij1[:][0]
v_i2 = v_ij1[:][1]
progress_rate2(v_ij1, v_ij2, [1.0,2.0,1.0], 10)

[40.0, 10.0]

### Problem 5

In [80]:
def reaction_rate(v_ij1, v_ij2, x, k):
    """Returns the reaction rate for a system of reactions of the following form:
        (v_11')A + (v_21')B --> (v_31'')C
        (v_32')C --> (v_12'')A + (v_22'')B 
        
    INPUTS
    =======
    v_ij1: a numpy array of floats, representing the stoichiometric coefficient of reactnat i in reaction j
    v_ij2: a numpy array of floats, representing the stoichiometric coefficient of product i in reactio j
    x: a list of floats representing the concentration of each reactant and product
    k: float, reaction rate coefficient
    
    RETURNS
    =======
    reaction rate for the system of reactions: a list of floats [f1, f2, ..., fi]
        fj is the reaction rate for specie i
    
    EXAMPLES:
    >>> reaction_rate(np.array([[1.0, 2.0, 0.0], [0.0, 0.0, 2.0]]), np.array([[0.0, 0.0, 1.0], [1.0, 2.0, 0.0]]),
                      [1.0,2.0,1.0], 10)
    [-30.0, -60.0, 20.0]

"""
    import numpy as np
    v_ij = v_ij2 - v_ij1
    vA = list(v_ij[:,0])
    vB = list(v_ij[:,1])
    vC = list(v_ij[:,2])
    w1 = progress_rate2(v_ij1, v_ij2, x, k)[0]
    w2 = progress_rate2(v_ij1, v_ij2, x, k)[1]
    f1 = vA[0]*w1 + vA[1]*w2
    f2 = vB[0]*w1 + vB[1]*w2
    f3 = vC[0]*w1 + vC[1]*w2
    f = [f1,f2,f3]
    return f

In [81]:
v_ij1 = np.array([[1.0, 2.0, 0.0], 
                 [0.0, 0.0, 2.0]])
v_ij2 = np.array([[0.0, 0.0, 1.0], 
                 [1.0, 2.0, 0.0]])

reaction_rate(v_ij1, v_ij2, [1.0,2.0,1.0], 10)

[[-1. -2.  1.]
 [ 1.  2. -2.]]
[-1.0, 1.0]
40.0 10.0


[-30.0, -60.0, 20.0]

### Problem 6

In [ ]:
%% file chemkin.py


In [8]:
# test
def k_arr(A, E, T, R=8.314):
    import numpy as np
    if A <= 0:
        raise ValueError("The Arrhenius prefactor must be strictly positive.")
    if T <= 0:
        raise ValueError("Temperature must be strictly positive.")
    k2 = A * np.exp(-E / (R*T))
    return k2

In [13]:
k2 = k_arr(10**7, 10**3, 10**2)
print(k2)

3003549.08896


In [14]:
def k_modarr(A, b, E, T, R=8.314):
    import numpy as np
    if A <= 0:
        raise ValueError("The Arrhenius prefactor must be strictly positive.")
    if T <= 0:
        raise ValueError("Temperature must be strictly positive.")
    if np.iscomplex(b):
        raise ValueError("The parameter b must be real.")
    k3 = A * (T**b) * np.exp(-E / (R*T))
    return k3

In [17]:
k_modarr(10**7, 0.5, 10**3, 10**2)

30035490.889639609

In [22]:
import sys
print(sys.float_info.max)
print(sys.float_info.min)

1.7976931348623157e+308
2.2250738585072014e-308
